In [1]:
## Import All Needed Libraries

import os
import gym
import ray
from animalai.envs.arena_config import ArenaConfig
from animalai.envs.gym.environment import AnimalAIGym
from ray.rllib.agents import ppo

from ray.tune import register_env, tune

from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.models import ModelCatalog
from ray.tune.logger import pretty_print

from cache_model import *
from config import get_cfg
from custom_model import *

Instructions for updating:
non-resource variables are not supported in the long term
2021-04-02 18:55:32,045	WARNING deprecation.py:34 -- DeprecationWarning: `ray.rllib.env.atari_wrappers....` has been deprecated. Use `ray.rllib.env.wrappers.atari_wrappers....` instead. This will raise an error in the future!


In [2]:
## Reuse Wrapper for AnimalAI Environment

class UnityEnvWrapper(gym.Env):
    def __init__(self, env_config):
        self.vector_index = env_config.vector_index
        self.worker_index = env_config.worker_index
        self.worker_id = env_config["unity_worker_id"] + env_config.worker_index
        self.env = AnimalAIGym(
            environment_filename = "../examples/env/AnimalAI",
            worker_id = self.worker_id,
            flatten_branched = True,
            uint8_visual = True,
            arenas_configurations = ArenaConfig(env_config['arena_to_train'])
        )
        self.action_space = self.env.action_space
        self.observation_space = self.env.observation_space
        
    def reset(self):
        return self.env.reset()
    
    def step(self, action):
        return self.env.step(action)

In [8]:
## Setup configuration to use

conf = {
    "num_workers": 0,
    "env_config": {
        "unity_worker_id": 60,
        "arena_to_train": '../examples/configurations/curriculum/0.yml'
        },
        "model": {
            "custom_model": 'my_cnn_rnn_model',
            "custom_model_config": {},
        },
        "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "1")),
        "num_workers": 1,  # parallelism
        "framework": "torch",
        "train_batch_size": 500
       }
conf

{'num_workers': 1,
 'env_config': {'unity_worker_id': 60,
  'arena_to_train': '../examples/configurations/curriculum/0.yml'},
 'model': {'custom_model': 'my_cnn_rnn_model', 'custom_model_config': {}},
 'num_gpus': 1,
 'framework': 'torch',
 'train_batch_size': 500}

In [9]:
## Setup and register environment
ray.shutdown()
ray.init()

# Register custom models so that we can give the ID to the policy trainer
# ModelCatalog.register_custom_model("my_fc_model", MyFCForwardModel)
ModelCatalog.register_custom_model("my_rnn_model", MyRNNModel)
ModelCatalog.register_custom_model("my_convgru_model", MyConvGRUModel)  # NOTE: Only works with image observations.
ModelCatalog.register_custom_model("my_cnn_rnn_model", MyCNNRNNModel)

register_env("unity_env", lambda config: UnityEnvWrapper(config))

2021-04-02 18:58:43,488	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265


In [10]:
## Setup trainer
trainer = PPOTrainer(config=conf, env= "unity_env")
checkpoint = '/home/azibit/ray_results/PPO_unity_env_2021-03-30_12-13-54487__wig/checkpoint_410/checkpoint-410'
trainer.restore(checkpoint)

(pid=12070) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=12070) Instructions for updating:
(pid=12070) non-resource variables are not supported in the long term
(pid=12070) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=12070) Instructions for updating:
(pid=12070) non-resource variables are not supported in the long term


(pid=12070) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=12070) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=12070) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=12070) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=12070) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=12070) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=12070) Preloaded 'lib_burst_generated.so'
(pid=12070) Preloaded 'libgrpc_csharp_ext.x64.so'
(pid=12070) Preloaded 'ScreenSelector.so'
(pid=12070) Display 0 '0': 1920x1080 (primary device).
(pid=12070) Logging to /home/azibit/.config/unity3d/Unity Technologies/UnityEnvironment/Player.log
(pid=12070)

(pid=12070) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=12070) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=12070) INFO:mlagents_envs:Connected new brain:
(pid=12070) AnimalAI?team=0
(pid=12070) INFO:gym_unity:1 agents within environment.
(pid=12070) 2021-04-02 18:58:52,943	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=12070) INFO:mlagents_envs:Connected new brain:
(pid=12070) AnimalAI?team=0
(pid=12070) INFO:gym_unity:1 agents within environment.
(pid=12070) 2021-04-02 18:58:52,943	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=12070) 2021-04-02 18:58:52,979	WARNING deprecation.py:34 -- Deprecati

FileNotFoundError: [Errno 2] No such file or directory: '/home/azibit/ray_results/PPO_unity_env_2021-03-31_13-35-37mieqkstg/checkpoint_202.tune_metadata'

In [7]:
env = AnimalAIGym(
            environment_filename = "../examples/env/AnimalAI",
            worker_id = 132,
            flatten_branched = True,
            uint8_visual = True,
            arenas_configurations = ArenaConfig('../examples/configurations/curriculum/2_inside.yml')
)

INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
INFO:mlagents_envs:Connected new brain:
AnimalAI?team=0
INFO:gym_unity:1 agents within environment.


In [12]:
agent_groups = env._env.get_agent_groups()
agent_groups

['AnimalAI?team=0']

In [21]:
# run for number of steps
episode_reward = 0
done = False
recurrent_initial_state = trainer.get_policy().model.get_initial_state()
obs = env.reset()
agent_group = agent_groups[0]

visual_observations = []

velocity_observations = []
rewards = []


while not done:
#     action = np.array(actions[step]).reshape(1, 2)
    action = trainer.compute_action(obs, recurrent_initial_state)
    action, recurrent_initial_state = action[0], action[1]

#     env._env.set_actions(agent_group = agent_group, action = action)
#     env._env.step()
    
    step_result = env.step(action)
    
    
    visual_observations.append(step_result)
    velocity_observations.append(step_result)
    
    done = step_result.done[0]
    rewards.append(step_result.reward[0])
    
    max_step_reached = step_result.max_step[0]
    step += 1
    
#         # print(obs.shape)
#     while not done:
#         action = trainer.compute_action(obs, recurrent_initial_state)
#         action, recurrent_initial_state = action[0], action[1]
#     #     action = trainer.compute_action(obs)
#         print("ACTION: ", action)
#     #     action, recurrent_initial_state = action[0], action[1]
#         obs, reward, done, info = env.step(action)

#         print("REWARD: ", reward)
#     #     print(info)
#         episode_reward += reward

AttributeError: 'tuple' object has no attribute 'done'

In [ ]:
## Train the model
checkpoint = ""
for i in range(3):
   # Perform one iteration of training the policy with PPO
   result = trainer.train()
   print(pretty_print(result))

   if i % 2 == 0:
       checkpoint = trainer.save()
       print("checkpoint saved at", checkpoint)

In [48]:
checkpoint

'/home/azibit/ray_results/PPO_unity_env_2021-03-28_18-18-42y6oxjq7g/checkpoint_3/checkpoint-3'

In [ ]:
trainer = ppo.PPOTrainer(config=conf, env='unity_env')
trainer.restore(checkpoint)

In [15]:
ray.shutdown()

In [80]:
# arena_configurations = ['0.yml', '1.yml', '2.yml', '3.yml', '4.yml']
arena_configurations = ['0.yml', '1.yml']

result_per_arena = [0.8, 0.8, 0.8, 0.6, 0.2]
MIN_NUMBER_OF_EPISODES = 100
checkpoint = ""

for item in range(len(arena_configurations)):
    
    ## Setup configuration to use
    conf = {
        "num_workers": 0,
        "env_config": {
            "unity_worker_id": 60,
            "arena_to_train": '../examples/configurations/curriculum/' + arena_configurations[item],
            },
            "model": {
                "custom_model": 'my_cnn_rnn_model',
                "custom_model_config": {},
            },
            "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "2")),
            "num_workers": 1,  # parallelism
            "framework": "torch",
            "train_batch_size": 500
           }
    
    
    ## Setup and register environment
    ray.shutdown()
    ray.init()

    # Register custom models so that we can give the ID to the policy trainer
    # ModelCatalog.register_custom_model("my_fc_model", MyFCForwardModel)
    ModelCatalog.register_custom_model("my_rnn_model", MyRNNModel)
    ModelCatalog.register_custom_model("my_convgru_model", MyConvGRUModel)  # NOTE: Only works with image observations.
    ModelCatalog.register_custom_model("my_cnn_rnn_model", MyCNNRNNModel)

    register_env("unity_env", lambda config: UnityEnvWrapper(config))
    
    ## Setup trainer
    trainer = PPOTrainer(config=conf, env= "unity_env")
    
    ## Loading and restoring a trained agent from a checkpoint 
    if checkpoint:    
        trainer.restore(checkpoint)
    
    result = trainer.train()
    min_episodes = MIN_NUMBER_OF_EPISODES
    
    ## Train the model until performance improves to a certain threshold
    while result['episode_reward_mean'] < result_per_arena[item] and min_episodes > 0:
        result = trainer.train()
        min_episodes -= 1

    checkpoint = trainer.save()
    print("checkpoint saved at", checkpoint)
    print(pretty_print(result))

2021-03-31 12:11:50,802	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
(pid=123254) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=123254) Instructions for updating:
(pid=123254) non-resource variables are not supported in the long term
(pid=123254) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=123254) Instructions for updating:
(pid=123254) non-resource variables are not supported in the long term
(pid=123254) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variabl

(pid=123254) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=123254) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=123254) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=123254) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=123254) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=123254) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=123254) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=123254) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=123254) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey

(pid=123254) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=123254) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=123254) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=123254) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=123254) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=123254) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=123254) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=123254) INFO:mlagents_envs:Connected new brain:
(pid=123254) AnimalAI?t

checkpoint saved at /home/azibit/ray_results/PPO_unity_env_2021-03-31_12-11-51yp_j3tj1/checkpoint_101/checkpoint-101
custom_metrics: {}
date: 2021-03-31_13-35-34
done: false
episode_len_mean: 212.76
episode_reward_max: 0.6016666842624545
episode_reward_mean: -0.5009499412681907
episode_reward_min: -0.9999999310821295
episodes_this_iter: 3
episodes_total: 306
experiment_id: 2075e209d6be448db19d9b920ede4363
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 3.4171875
      cur_lr: 5.0e-05
      entropy: 0.7176802635192872
      entropy_coeff: 0.0
      kl: 0.011574807204306125
      policy_loss: -0.12225581109523773
      total_loss: -0.0812902458012104
      vf_explained_var: 0.9841369390487671
      vf_loss: 0.0014122813008725643
  num_steps_sampled: 60600
  num_steps_trained: 60600
iterations_since_restore: 101
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 17.5
  ram

2021-03-31 13:35:36,876	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
(pid=130418) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=130418) Instructions for updating:
(pid=130418) non-resource variables are not supported in the long term
(pid=130418) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=130418) Instructions for updating:
(pid=130418) non-resource variables are not supported in the long term
(pid=130418) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variabl

(pid=130418) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=130418) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=130418) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=130418) Preloaded 'lib_burst_generated.so'
(pid=130418) Preloaded 'libgrpc_csharp_ext.x64.so'
(pid=130418) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=130418) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=130418) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=130418) Preloaded 'lib_burst_generated.so'
(pid=130418) Preloaded 'libgrpc_csharp_ext.x64.so'
(pid=130418) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86

(pid=130418) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=130418) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=130418) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=130418) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=130418) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=130418) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=130418) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=130418) INFO:mlagents_envs:Connected to Unity environment with package 

checkpoint saved at /home/azibit/ray_results/PPO_unity_env_2021-03-31_13-35-37mieqkstg/checkpoint_202/checkpoint-202
custom_metrics: {}
date: 2021-03-31_15-13-56
done: false
episode_len_mean: 250.0
episode_reward_max: -0.9999999310821295
episode_reward_mean: -0.9999999310821295
episode_reward_min: -0.9999999310821295
episodes_this_iter: 2
episodes_total: 548
experiment_id: 2075e209d6be448db19d9b920ede4363
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 3.4171875
      cur_lr: 5.0e-05
      entropy: 1.2816949844360352
      entropy_coeff: 0.0
      kl: 0.012957278452813626
      policy_loss: -0.19882270991802214
      total_loss: -0.15444277115166188
      vf_explained_var: 0.9886918067932129
      vf_loss: 0.00010248147955280729
  num_steps_sampled: 121200
  num_steps_trained: 121200
iterations_since_restore: 101
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 20.395

In [7]:
trainer.get_policy().model

MyCNNRNNModel(
  (cnn): Sequential(
    (0): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (linear): Linear(in_features=3136, out_features=64, bias=True)
  (lstm): LSTM(64, 256, batch_first=True)
  (action_branch): Linear(in_features=256, out_features=9, bias=True)
  (value_branch): Linear(in_features=256, out_features=1, bias=True)
)

In [36]:
## Trying to test the trained agent on an environment

env = AnimalAIGym(
            environment_filename = "../examples/env/AnimalAI",
            worker_id = 132,
            flatten_branched = True,
            uint8_visual = True,
            arenas_configurations = ArenaConfig('../examples/configurations/curriculum/2_inside.yml')
)

INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
INFO:mlagents_envs:Connected new brain:
AnimalAI?team=0
INFO:gym_unity:1 agents within environment.


In [37]:
# run for number of steps
episode_reward = 0
done = False
recurrent_initial_state = trainer.get_policy().model.get_initial_state()
obs = env.reset()


# print(obs.shape)
while not done:
    action = trainer.compute_action(obs, recurrent_initial_state)
    action, recurrent_initial_state = action[0], action[1]
#     action = trainer.compute_action(obs)
    print("ACTION: ", action)
#     action, recurrent_initial_state = action[0], action[1]
    obs, reward, done, info = env.step(action)
    
    print("REWARD: ", reward)
#     print(info)
    episode_reward += reward

ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  2
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  7
REWARD:  

ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  2
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  

REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  2
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000

REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000

REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  2
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  2
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000

...

ACTION:  5
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  8
REWARD:  

ACTION:  6
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  7
REWARD:  

ACTION:  0
REWARD:  -0.00020000001
ACTION:  2
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  2
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  3
REWARD:  

ACTION:  3
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  2
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  2
REWARD:  -0.00020000001
ACTION:  3
REWARD:  

REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  2
REWARD:  -0.00020000001
ACTION:  5
REWARD:  -0.00020000001
ACTION:  2
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000001
ACTION:  8
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  1
REWARD:  -0.00020000001
ACTION:  7
REWARD:  -0.00020000001
ACTION:  2
REWARD:  -0.00020000001
ACTION:  6
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  4
REWARD:  -0.00020000001
ACTION:  0
REWARD:  -0.00020000001
ACTION:  3
REWARD:  -0.00020000

In [38]:
episode_reward

-1.0000000474974513

In [25]:
episode_reward

-0.9999999310821295

In [7]:
env.close()

NameError: name 'env' is not defined

In [65]:
arena_configurations = ['0.yml', '1.yml', '2.yml']
result_per_arena = [0.8, 0.9, 0.7]
checkpoint = ""
number_of_training_per_arena = 2

for arena_file in arena_configurations:
    
    ## Setup configuration to use
    conf = {
        "num_workers": 0,
        "env_config": {
            "unity_worker_id": 57,
            "arena_to_train": '../examples/configurations/curriculum/' + arena_file,
            },
            "model": {
                "custom_model": 'my_cnn_rnn_model',
                "custom_model_config": {},
            },
            "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
            "num_workers": 1,  # parallelism
            "framework": "torch",
            "train_batch_size": 500
           }
    
    
    ## Setup and register environment
    ray.shutdown()
    ray.init()

    # Register custom models so that we can give the ID to the policy trainer
    # ModelCatalog.register_custom_model("my_fc_model", MyFCForwardModel)
    ModelCatalog.register_custom_model("my_rnn_model", MyRNNModel)
    ModelCatalog.register_custom_model("my_convgru_model", MyConvGRUModel)  # NOTE: Only works with image observations.
    ModelCatalog.register_custom_model("my_cnn_rnn_model", MyCNNRNNModel)

    register_env("unity_env", lambda config: UnityEnvWrapper(config))
    
    ## Setup trainer
    trainer = PPOTrainer(config=conf, env= "unity_env")
    
    ## Loading and restoring a trained agent from a checkpoint 
    if checkpoint:    
        trainer.restore(checkpoint)
    
    ## Train the model until performance improves to a certain threshold
    for i in range(number_of_training_per_arena):
       i+=1 
       # Perform one iteration of training the policy with PPO
       result = trainer.train()

       if i % number_of_training_per_arena == 0:
           checkpoint = trainer.save()
           print("checkpoint saved at", checkpoint)
           print(pretty_print(result))

2021-03-28 18:44:32,337	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8266
(pid=439566) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=439566) Instructions for updating:
(pid=439566) non-resource variables are not supported in the long term
(pid=439566) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=439566) Instructions for updating:
(pid=439566) non-resource variables are not supported in the long term
(pid=439566) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variabl

(pid=439566) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=439566) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=439566) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=439566) Preloaded 'lib_burst_generated.so'
(pid=439566) Preloaded 'libgrpc_csharp_ext.x64.so'
(pid=439566) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=439566) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=439566) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=439566) Preloaded 'lib_burst_generated.so'
(pid=439566) Preloaded 'libgrpc_csharp_ext.x64.so'
(pid=439566) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86

(pid=439566) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=439566) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=439566) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=439566) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=439566) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=439566) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=439566) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=439566) INFO:mlagents_envs:Connected to Unity environment with package 

2021-03-28 18:44:38,998	WARNING util.py:47 -- Install gputil for GPU system monitoring.
(pid=439566) 2021-03-28 18:44:39,008	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=439566) 2021-03-28 18:44:39,008	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=439566) 2021-03-28 18:44:39,008	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=439566) 2021-03-28 18:44:39,008	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=439566) 2021-03-28 18:44:39,008	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This wi

checkpoint saved at /home/azibit/ray_results/PPO_unity_env_2021-03-28_18-44-33hqas1fk1/checkpoint_2/checkpoint-2
custom_metrics: {}
date: 2021-03-28_18-45-22
done: false
episode_len_mean: 163.5
episode_reward_max: 0.9350000089034438
episode_reward_mean: -0.15405550071348748
episode_reward_min: -0.9999999310821295
episodes_this_iter: 3
episodes_total: 6
experiment_id: 81b5de88f1d845ed8ae6fd084b09157c
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 0.30000000000000004
      cur_lr: 5.0e-05
      entropy: 2.1149948239326477
      entropy_coeff: 0.0
      kl: 0.028780032880604267
      policy_loss: -0.05568232759833336
      total_loss: -0.023151567205786705
      vf_explained_var: 0.4177202880382538
      vf_loss: 0.02389675984159112
  num_steps_sampled: 1200
  num_steps_trained: 1200
iterations_since_restore: 2
node_ip: 10.26.177.163
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 32.42666

...

2021-03-28 18:46:25,277	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8266
(pid=440366) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=440366) Instructions for updating:
(pid=440366) non-resource variables are not supported in the long term
(pid=440366) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=440366) Instructions for updating:
(pid=440366) non-resource variables are not supported in the long term
(pid=440366) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variabl

(pid=440366) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=440366) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=440366) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=440366) Preloaded 'lib_burst_generated.so'
(pid=440366) Preloaded 'libgrpc_csharp_ext.x64.so'
(pid=440366) Preloaded 'ScreenSelector.so'
(pid=440366) Display 0 '0': 1920x1080 (primary device).
(pid=440366) Display 1 'C32F391 32"': 1920x1080 (secondary device).
(pid=440366) Logging to /home/azibit/.config/unity3d/Unity Technologies/UnityEnvironment/Player.log
(pid=440366) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=440366) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=440366) Mono config path = '/home/azibit/COMS_673/AnimalAI-

(pid=440366) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=440366) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=440366) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=440366) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=440366) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=440366) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=440366) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=440366) INFO:mlagents_envs:Connected to Unity environment with package 

2021-03-28 18:46:32,282	WARNING util.py:47 -- Install gputil for GPU system monitoring.
2021-03-28 18:46:32,298	INFO trainable.py:372 -- Restored on 10.26.177.163 from checkpoint: /home/azibit/ray_results/PPO_unity_env_2021-03-28_18-45-26p12bgvjp/checkpoint_4/checkpoint-4
2021-03-28 18:46:32,299	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 4, '_timesteps_total': None, '_time_total': 93.52463006973267, '_episodes_total': 11}
(pid=440366) 2021-03-28 18:46:32,322	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=440366) 2021-03-28 18:46:32,322	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=440366) 2021-03-28 18:46:32,322	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an 

checkpoint saved at /home/azibit/ray_results/PPO_unity_env_2021-03-28_18-46-26rufuvxun/checkpoint_6/checkpoint-6
custom_metrics: {}
date: 2021-03-28_18-47-16
done: false
episode_len_mean: 250.0
episode_reward_max: -0.9999999310821295
episode_reward_mean: -0.9999999310821295
episode_reward_min: -0.9999999310821295
episodes_this_iter: 2
episodes_total: 15
experiment_id: 81b5de88f1d845ed8ae6fd084b09157c
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 0.30000000000000004
      cur_lr: 5.0e-05
      entropy: 2.093348801136017
      entropy_coeff: 0.0
      kl: 0.02126031508669257
      policy_loss: -0.30984465032815933
      total_loss: -0.3008092902600765
      vf_explained_var: 0.6789491176605225
      vf_loss: 0.0026572615606710315
  num_steps_sampled: 3600
  num_steps_trained: 3600
iterations_since_restore: 2
node_ip: 10.26.177.163
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 27.789655

In [74]:
## Setup configuration to use

config = {
    "num_workers": 0,
    "env_config": {
        "unity_worker_id": 57,
        "arena_to_train": '../examples/configurations/curriculum/0.yml',
        },
        "model": {
            "custom_model": 'my_cnn_rnn_model',
            "custom_model_config": {},
        },
        "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
        "num_workers": 1,  # parallelism
        "framework": "torch",
        "train_batch_size": 500
       }

In [77]:
env = AnimalAIGym(
            environment_filename = "../examples/env/AnimalAI",
            worker_id = 123,
            flatten_branched = True,
            uint8_visual = True,
            arenas_configurations = ArenaConfig('../examples/configurations/curriculum/0.yml')
        )

INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
INFO:mlagents_envs:Connected new brain:
AnimalAI?team=0
INFO:gym_unity:1 agents within environment.


(84, 84, 3)


IndexError: list index out of range

In [82]:
trainer.get_policy().model

MyCNNRNNModel(
  (cnn): Sequential(
    (0): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (linear): Linear(in_features=3136, out_features=64, bias=True)
  (lstm): LSTM(64, 256, batch_first=True)
  (action_branch): Linear(in_features=256, out_features=9, bias=True)
  (value_branch): Linear(in_features=256, out_features=1, bias=True)
)

AttributeError: 'PPO' object has no attribute 'result'